In [26]:
#house price detection basic model
import torch
import pandas as pd
import numpy
from sklearn.preprocessing import MinMaxScaler

df_train=pd.read_csv("/content/sample_data/california_housing_train.csv")
print(df_train)
features=["housing_median_age","total_rooms","total_bedrooms","population","households","median_income"]
inp_dim=len(features)
print(inp_dim)

out_dim=1
num_layers=4
batch_size=16

x_train=df_train[features].values
#scaling the values
scaler=MinMaxScaler()
x_train=scaler.fit_transform(x_train)
x_train=torch.from_numpy(x_train).float()
y_train=df_train["median_house_value"].values
y_train=y_train.reshape(-1,1)
y_train=scaler.fit_transform(y_train)
y_train=torch.from_numpy(y_train).float()
print(x_train.shape)
print(y_train)
print(y_train.shape)


       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0        -114.31     34.19                15.0       5612.0          1283.0   
1        -114.47     34.40                19.0       7650.0          1901.0   
2        -114.56     33.69                17.0        720.0           174.0   
3        -114.57     33.64                14.0       1501.0           337.0   
4        -114.57     33.57                20.0       1454.0           326.0   
...          ...       ...                 ...          ...             ...   
16995    -124.26     40.58                52.0       2217.0           394.0   
16996    -124.27     40.69                36.0       2349.0           528.0   
16997    -124.30     41.84                17.0       2677.0           531.0   
16998    -124.30     41.80                19.0       2672.0           552.0   
16999    -124.35     40.54                52.0       1820.0           300.0   

       population  households  median_income  media

In [27]:
#test data
df_test=pd.read_csv("/content/sample_data/california_housing_test.csv")
print(df_test.head)
x_test=df_test[features].values
x_test=scaler.fit_transform(x_test)
x_test=torch.from_numpy(x_test).float()
y_test=df_test["median_house_value"].values
y_test=y_test.reshape(-1,1)
y_test=scaler.fit_transform(y_test)
y_test=torch.from_numpy(y_test).float()
print(x_test.shape)
print(y_test.shape)

train_unsplit_data=torch.utils.data.TensorDataset(x_train,y_train)
ratio=0.8
split_index=int(len(x_train)*ratio)
x_train,x_val=x_train[:split_index],x_train[split_index:]
y_train,y_val=y_train[:split_index],y_train[split_index:]
train_dataset=torch.utils.data.TensorDataset(x_train,y_train)
val_dataset=torch.utils.data.TensorDataset(x_val,y_val)
val_loader=torch.utils.data.DataLoader(val_dataset,batch_size,shuffle=True)
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size,shuffle=True)
test_dataset=torch.utils.data.TensorDataset(x_test,y_test)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size,shuffle=True)

<bound method NDFrame.head of       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0       -122.05     37.37                27.0       3885.0           661.0   
1       -118.30     34.26                43.0       1510.0           310.0   
2       -117.81     33.78                27.0       3589.0           507.0   
3       -118.36     33.82                28.0         67.0            15.0   
4       -119.67     36.33                19.0       1241.0           244.0   
...         ...       ...                 ...          ...             ...   
2995    -119.86     34.42                23.0       1450.0           642.0   
2996    -118.14     34.06                27.0       5257.0          1082.0   
2997    -119.70     36.30                10.0        956.0           201.0   
2998    -117.12     34.10                40.0         96.0            14.0   
2999    -119.63     34.42                42.0       1765.0           263.0   

      population  households  med

In [28]:
#making the regression model now
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)
class Housing(nn.Module):
    def __init__(self,inp_dim,out_dim,num_layers,batch_size):
        super(Housing,self).__init__()
        self.inp_dim=inp_dim
        self.out_dim=out_dim
        self.num_layers=out_dim
        self.batch_size=batch_size
        self.layer1=nn.Linear(inp_dim,32)#32 neurons in the 1st hidden layer
        self.layer2=nn.Linear(32,16)#16 neurons in the 2nd hidden layer
        self.layer3=nn.Linear(16,1)
    def forward(self,x):
        x=F.relu(self.layer1(x))
        x=F.relu(self.layer2(x))
        output=self.layer3(x)
        return output

epochs=50
lr=0.001
lossfunc=nn.MSELoss()
model=Housing(inp_dim,out_dim,num_layers,batch_size)
optimizer=optim.Adam(model.parameters(),lr)
print("Training started")

for epoch in range(epochs):
    model.train()
    train_loss=0.0
    for inp,out in train_loader:
        optimizer.zero_grad()
        output=model(inp)
        loss_val=lossfunc(output,out)
        loss_val.backward()
        optimizer.step()
        train_loss+=loss_val.item()

    model.eval()
    val_loss=0.0
    with torch.no_grad():
        for inp,out in val_loader:
            output=model(inp)
            val_loss+=lossfunc(output,out).item()
    print(f"Epoch{epoch},Train Loss:{train_loss:.4f},Val Loss:{val_loss:.4f}")


cpu
Training started
Epoch0,Train Loss:35.8228,Val Loss:6.3657
Epoch1,Train Loss:20.8462,Val Loss:6.0479
Epoch2,Train Loss:20.0360,Val Loss:5.8534
Epoch3,Train Loss:19.4632,Val Loss:6.1753
Epoch4,Train Loss:19.0336,Val Loss:5.3435
Epoch5,Train Loss:18.5201,Val Loss:5.5727
Epoch6,Train Loss:18.3453,Val Loss:5.1809
Epoch7,Train Loss:17.7836,Val Loss:5.5347
Epoch8,Train Loss:17.5788,Val Loss:4.9413
Epoch9,Train Loss:17.3208,Val Loss:5.0782
Epoch10,Train Loss:17.1957,Val Loss:4.7423
Epoch11,Train Loss:16.9564,Val Loss:4.7239
Epoch12,Train Loss:16.7746,Val Loss:4.8020
Epoch13,Train Loss:16.7599,Val Loss:4.9801
Epoch14,Train Loss:16.5920,Val Loss:4.6712
Epoch15,Train Loss:16.4255,Val Loss:4.8144
Epoch16,Train Loss:16.2588,Val Loss:5.0935
Epoch17,Train Loss:16.2872,Val Loss:4.6160
Epoch18,Train Loss:16.1800,Val Loss:4.5575
Epoch19,Train Loss:16.0817,Val Loss:4.5618
Epoch20,Train Loss:16.0350,Val Loss:4.5699
Epoch21,Train Loss:15.9454,Val Loss:4.5708
Epoch22,Train Loss:15.9446,Val Loss:4.7635


In [54]:
sample_data = torch.tensor([[15, 200, 1283, 1015, 472, 1.493600]], dtype=torch.float32)

input_scaled_np = scaler.transform(sample_data.numpy())
input_scaled = torch.tensor(input_scaled_np, dtype=torch.float32).to(device)


model.eval()
with torch.no_grad():
    pred_price = model(input_scaled).cpu().numpy()

print(f"Predicted median house value: ${pred_price[0][0]:,.2f}")


Predicted median house value: $718.05
